In [1]:
#Authors Amulya J N  ojaswini2004@gmail.com , Nikitha M N  nikitha.2723@gmail.com

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from keras.optimizers import Adam
import folium
from folium.plugins import AntPath
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog
import webbrowser

# Load the dataset
file_path = r'E:\Amulya B-Tech\Code\FINAL\a.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(data.head())

# Check for missing values
print(data.isnull().sum())

# Drop rows with missing values if any
data.dropna(inplace=True)

# Convert 'Date' to datetime
data['Date'] = pd.to_datetime(data['Date'], format='%d-%b-%y')

# Convert 'Time' to seconds since midnight
data['Time'] = pd.to_timedelta(data['Time'] + ':00').dt.total_seconds()

# Function to prepare data
def prepare_data(train_start_date, train_end_date, test_start_date, test_end_date):
    # Filter data for specific training dates
    train_data = data[(data['Date'] >= train_start_date) & (data['Date'] <= train_end_date)]

    # Filter data for specific testing dates
    test_data = data[(data['Date'] >= test_start_date) & (data['Date'] <= test_end_date)]

    # Normalize 'Latitude', 'Longitude', and 'Time' separately for training data
    scalers = {
        'Latitude': MinMaxScaler(),
        'Longitude': MinMaxScaler(),
        'Time': MinMaxScaler()
    }

    for feature in scalers.keys():
        train_data[[feature]] = scalers[feature].fit_transform(train_data[[feature]])

    # Normalize 'Latitude', 'Longitude', and 'Time' for testing data using training scalers
    for feature in scalers.keys():
        test_data[[feature]] = scalers[feature].transform(test_data[[feature]])

    # Select features and target
    features = ['Latitude', 'Longitude', 'Time']
    target = ['Latitude', 'Longitude']

    # Prepare sequences for LSTM
    def create_sequences(data, seq_length):
        X = []
        y = []
        for i in range(seq_length, len(data)):
            X.append(data[i-seq_length:i])
            y.append(data[i, 0:2])  # Predicting latitude and longitude
        return np.array(X), np.array(y)

    seq_length = 20  # Increase sequence length
    X_train, y_train = create_sequences(train_data[features].values, seq_length)
    X_test, y_test = create_sequences(test_data[features].values, seq_length)

    return X_train, y_train, X_test, y_test, scalers

# Function to build and train the model
def train_model(X_train, y_train):
    # Build the model
    model = Sequential()
    model.add(Bidirectional(LSTM(150, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(150, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(GRU(100, return_sequences=False)))
    model.add(Dropout(0.3))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(2))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, y_train, epochs=40, batch_size=32, validation_split=0.2)
    return model

# Function to make predictions, save the map, and save the predictions to a CSV file
def predict_and_save_map(model, X_test, y_test, scalers):
    # Make predictions
    y_pred = model.predict(X_test)

    # Rescale predictions
    y_test_rescaled = np.column_stack([
        scalers['Latitude'].inverse_transform(y_test[:, [0]]),
        scalers['Longitude'].inverse_transform(y_test[:, [1]]),
        scalers['Time'].inverse_transform(X_test[:, :, 2])[:, 0]  # Rescale time for color mapping
    ])

    y_pred_rescaled = np.column_stack([
        scalers['Latitude'].inverse_transform(y_pred[:, [0]]),
        scalers['Longitude'].inverse_transform(y_pred[:, [1]]),
        scalers['Time'].inverse_transform(X_test[:, :, 2])[:, 0]  # Rescale time for color mapping
    ])

    # Save predictions to CSV
    predictions_df = pd.DataFrame({
        'Actual Latitude': y_test_rescaled[:, 0],
        'Actual Longitude': y_test_rescaled[:, 1],
        'Predicted Latitude': y_pred_rescaled[:, 0],
        'Predicted Longitude': y_pred_rescaled[:, 1]
    })
    predictions_df.to_csv('predictions.csv', index=False)

    # Center the map around the mean latitude and longitude
    center_lat = np.mean(y_test_rescaled[:, 0])
    center_lng = np.mean(y_test_rescaled[:, 1])

    # Create a folium map
    m = folium.Map(location=[center_lat, center_lng], zoom_start=14)

    # Add actual trajectory using AntPath
    actual_coords = list(zip(y_test_rescaled[:, 0], y_test_rescaled[:, 1]))
    AntPath(actual_coords, color='blue', weight=2.5, opacity=0.8).add_to(m)

    # Add predicted trajectory using AntPath
    pred_coords = list(zip(y_pred_rescaled[:, 0], y_pred_rescaled[:, 1]))
    AntPath(pred_coords, color='red', weight=2.5, opacity=0.8).add_to(m)

    # Add markers at the start points
    folium.Marker(location=actual_coords[0], popup='Actual Start', icon=folium.Icon(color='blue')).add_to(m)
    folium.Marker(location=pred_coords[0], popup='Predicted Start', icon=folium.Icon(color='red')).add_to(m)

    # Save the map to an HTML file and open it
    map_file = 'trajectory_map.html'
    m.save(map_file)
    webbrowser.open(map_file)

# Function to handle prediction process
def on_predict():
    try:
        train_start_date = start_date_entry.get()
        train_end_date = end_date_entry.get()
        test_start_date = test_start_date_entry.get()
        test_end_date = test_end_date_entry.get()
        
        X_train, y_train, X_test, y_test, scalers = prepare_data(train_start_date, train_end_date, test_start_date, test_end_date)
        model = train_model(X_train, y_train)
        predict_and_save_map(model, X_test, y_test, scalers)
        messagebox.showinfo("Success", "Prediction completed, map saved, and CSV file created!")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Create the main window
root = tk.Tk()
root.title("Trajectory Prediction")

# Create and place labels and entries
tk.Label(root, text="Training Start Date (YYYY-MM-DD):").grid(row=0, column=0)
start_date_entry = tk.Entry(root)
start_date_entry.grid(row=0, column=1)

tk.Label(root, text="Training End Date (YYYY-MM-DD):").grid(row=1, column=0)
end_date_entry = tk.Entry(root)
end_date_entry.grid(row=1, column=1)

tk.Label(root, text="Testing Start Date (YYYY-MM-DD):").grid(row=2, column=0)
test_start_date_entry = tk.Entry(root)
test_start_date_entry.grid(row=2, column=1)

tk.Label(root, text="Testing End Date (YYYY-MM-DD):").grid(row=3, column=0)
test_end_date_entry = tk.Entry(root)
test_end_date_entry.grid(row=3, column=1)

# Create and place the predict button
predict_button = tk.Button(root, text="Predict", command=on_predict)
predict_button.grid(row=4, column=0, columnspan=2)

# Run the GUI loop
root.mainloop()


   Latitude  Longitude  Elevation       Date   Time
0  11.67362   76.26272      879.0  01-Jan-19  06:00
1  11.67396   76.26302      878.0  01-Jan-19  07:00
2  11.67430   76.26332      875.0  01-Jan-19  08:00
3  11.67464   76.26362      879.0  01-Jan-19  09:00
4  11.67499   76.26393      888.0  01-Jan-19  10:00
Latitude     11
Longitude    11
Elevation    11
Date          0
Time          0
dtype: int64


C:\Users\Seema\AppData\Local\Temp\ipykernel_16960\1781935591.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[[feature]] = scalers[feature].transform(test_data[[feature]])
C:\Users\Seema\AppData\Local\Temp\ipykernel_16960\1781935591.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[[feature]] = scalers[feature].transform(test_data[[feature]])
C:\Users\Seema\AppData\Local\Temp\ipykernel_16960\1781935591.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

Epoch 1/40
191/191 ━━━━━━━━━━━━━━━━━━━━ 58s 174ms/step - loss: 0.0257 - val_loss: 6.7217e-04
Epoch 2/40
191/191 ━━━━━━━━━━━━━━━━━━━━ 31s 163ms/step - loss: 0.0029 - val_loss: 7.9352e-04
Epoch 3/40
191/191 ━━━━━━━━━━━━━━━━━━━━ 31s 160ms/step - loss: 0.0022 - val_loss: 6.0418e-04
Epoch 4/40
191/191 ━━━━━━━━━━━━━━━━━━━━ 31s 161ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 5/40
191/191 ━━━━━━━━━━━━━━━━━━━━ 30s 157ms/step - loss: 0.0016 - val_loss: 0.0010
Epoch 6/40
191/191 ━━━━━━━━━━━━━━━━━━━━ 30s 155ms/step - loss: 0.0015 - val_loss: 6.8144e-04
Epoch 7/40
191/191 ━━━━━━━━━━━━━━━━━━━━ 32s 167ms/step - loss: 0.0012 - val_loss: 4.6652e-04
Epoch 8/40
191/191 ━━━━━━━━━━━━━━━━━━━━ 31s 160ms/step - loss: 9.8341e-04 - val_loss: 3.6875e-04
Epoch 9/40
191/191 ━━━━━━━━━━━━━━━━━━━━ 31s 160ms/step - loss: 9.2259e-04 - val_loss: 2.3586e-04
Epoch 10/40
191/191 ━━━━━━━━━━━━━━━━━━━━ 31s 162ms/step - loss: 8.7568e-04 - val_loss: 3.9165e-04
Epoch 11/40
191/191 ━━━━━━━━━━━━━━━━━━━━ 30s 159ms/step - loss: 7